# 数据变换

## 数据指标的一致化处理

### 数据类型种类

数据类型一般有以下几种：

![Data Index](Data_Index.png)

### 数据类型的一致化处理方法

我们一般都选择将所有数据类型转化为极大型数据。

1. **极小型数据的转换**：
   - 这个转换方法针对的是极小型数据。原始数据 $ x $ 通过变换 $ x' = \frac{1}{x} $（对于 $ x > 0 $）进行转换。这相当于取其倒数。另外，为了让转换后的数据有一个明确的范围，也可以通过 $ x' = M - x $ 进一步转换，其中 $ M $ 是某个预定的最大值。

2. **中间型数据的转换**：
   - 此方法适用于介于两个极值之间的中间型数据。通过 $ x' = 1 - \frac{|x_i - x_{best}|}{\max |x_i - x_{best}|} $ 这个式子进行转换，其中 $ x_{best} $ 代表最优值。
3. **区间型数据的转换**：
   - 对于区间型数据，采用分段函数进行转换：
     $$
     x' = \begin{cases} 
     \frac{a - x}{c}, & x < a \\
     1, & a \leq x \leq b \\
     \frac{x - b}{c}, & x > b 
     \end{cases}
     $$
     其中 $ a $ 和 $ b $ 定义了中间区间，$ c $ 是一个用于缩放的常数，计算方法为 $ c = \max\{a - m, M - b\} $，$ M $ 和 $ m $ 分别是数据集中的最大值和最小值。

这些转换技术的目的是将不同类型的数据转换为具有可比性的形式，使其适合于统计分析或机器学习模型。每种方法都有其特定的应用场景和优势。例如，取倒数可以放大小数值的差异，而分段函数可以在保留数据结构的同时，对数据进行归一化。

## 数据指标的无量纲化处理

无量纲化是将不同量纲或量级的指标转换为无量纲的纯数值，以便进行合理比较。常用于决策分析中的TOPSIS法（Technique for Order Preference by Similarity to Ideal Solution，即理想解排序法）

### 无量纲化处理方法：

1. **线性变换法（最大-最小规范化）**:
   $$ b_{ij} = \frac{a_{ij} - \min_{i}(a_{ij})}{\max_{i}(a_{ij}) - \min_{i}(a_{ij})} $$
   这种方法将原始数据线性变换到[0, 1]区间内。

2. **反向线性变换法**:
   $$ b_{ij} = \frac{\max_{i}(a_{ij}) - a_{ij}}{\max_{i}(a_{ij}) - \min_{i}(a_{ij})} $$
   当原始数据越小越好时使用，也将数据变换到[0, 1]区间。

3. **区间无量纲化法**:
   对于不同的区间，使用不同的公式：
   - 当 $ a_{ij} $ 在 $ [a_{j}^{(1)}, a_{j}^{0}) $ 时：
        $$ b_{ij} = 1 - \frac{a_{j}^{0} - a_{ij}}{a_{j}^{0} - a_{j}^{(1)}} $$
   - 当 $ a_{ij} $ 在 $ [a_{j}^{0}, a_{j}^{*}] $ 时：
        $$ b_{ij} = 1 $$
   - 当 $ a_{ij} $ 在 $ (a_{j}^{*}, a_{j}^{(2)}] $ 时：
        $$ b_{ij} = 1 - \frac{a_{ij} - a_{j}^{*}}{a_{j}^{(2)} - a_{j}^{*}} $$
   - 否则：
        $$ b_{ij} = 0 $$
   其中 $ a_{j}^{(1)} $ 和 $ a_{j}^{(2)} $ 分别是指标 $ j $ 的容忍区间下限和上限，而 $ a_{j}^{0} $ 和 $ a_{j}^{*} $ 是优劣区间的下限和上限。

<p align = "center">
  <img src="Original_Data.png" alt="Original Data">
</p>

In [3]:
# 无量纲化处理
import numpy as np

def print_formatted(array, description):
    formatted_values = ", ".join(["{:.3f}".format(value) for value in array])
    print(f"{description}: [{formatted_values}]")

# 线性变换法
def min_max_normalization(array):
    return (array - np.min(array)) / (np.max(array) - np.min(array))

# 反向线性变换法
def reverse_min_max_normalization(array):
    return (np.max(array) - array) / (np.max(array) - np.min(array))

# 区间无量纲化法
def interval_normalization(array, optimal_interval, tolerance_interval):
    normalized_array = np.zeros_like(array)
    for i, value in enumerate(array):
        if tolerance_interval[0] <= value < optimal_interval[0]:
            normalized_array[i] = 1 - (optimal_interval[0] - value) / (optimal_interval[0] - tolerance_interval[0])
        elif optimal_interval[0] <= value <= optimal_interval[1]:
            normalized_array[i] = 1
        elif optimal_interval[1] < value <= tolerance_interval[1]:
            normalized_array[i] = 1 - (value - optimal_interval[1]) / (tolerance_interval[1] - optimal_interval[1])
        else:
            normalized_array[i] = 0
    return normalized_array

# 示例数据
A = np.array([
    [0.1, 0.2, 0.4, 0.9, 1.2],
    [5, 6, 7, 10, 2],
    [5000, 6000, 7000, 10000, 400],
    [4.7, 5.6, 6.7, 2.3, 1.8]
]).T  # 转置以匹配列格式

# 应用无量纲化方法
b1 = min_max_normalization(A[:, 0])
b2 = interval_normalization(A[:, 1], [5, 6], [2, 12])
b3 = reverse_min_max_normalization(A[:, 2])
b4 = reverse_min_max_normalization(A[:, 3])

# 输出结果
print_formatted(b1, "Min-Max Normalization")
print_formatted(b2, "Interval Normalization")
print_formatted(b3, "Reverse Min-Max Normalization")
print_formatted(b4, "Reverse Min-Max Normalization")

Min-Max Normalization: [0.000, 0.091, 0.273, 0.727, 1.000]
Interval Normalization: [1.000, 1.000, 0.833, 0.333, 0.000]
Reverse Min-Max Normalization: [0.521, 0.417, 0.312, 0.000, 1.000]
Reverse Min-Max Normalization: [0.408, 0.224, 0.000, 0.898, 1.000]


## 定性指标的量化处理



In [1]:
# 归一化，使用MinMaxScaler(按比例缩放，less recommended)
# K近邻算法
from sklearn.preprocessing import MinMaxScaler
from numpy import set_printoptions
from pandas import read_csv

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[ : , 0 : 8]
Y = array[ : , 8]
transformer = MinMaxScaler(feature_range = (0, 1))
newX = transformer.fit_transform(X)
set_printoptions(precision = 3)
print(newX)

[[0.353 0.744 0.59  ... 0.501 0.234 0.483]
 [0.059 0.427 0.541 ... 0.396 0.117 0.167]
 [0.471 0.92  0.525 ... 0.347 0.254 0.183]
 ...
 [0.294 0.608 0.59  ... 0.39  0.071 0.15 ]
 [0.059 0.633 0.492 ... 0.449 0.116 0.433]
 [0.059 0.467 0.574 ... 0.453 0.101 0.033]]
